In [45]:

import json
from pydantic import BaseModel, ValidationError, Field, validator
from typing import Optional, Any
# from geojson_length import calculate_distance, Unit
import requests

In [46]:
def string_to_bool_or_none(val:str):
    if not isinstance(val, str):
      return val

    if val == '':
      return None

    lowered_val = val.lower()

    if lowered_val == "false" or lowered_val == "usann":
        return False
    
    if lowered_val == "true" or lowered_val == "sann":
        return True

    return val


In [47]:
r = requests.get("http://localhost:8055/items/routes?limit=-1&fields=*.*")
routes = r.json()['data']
routes


[{'id': 0,
  'date_created': '2023-04-15T06:40:21.723Z',
  'date_updated': None,
  'aspect': ['S', 'Ø'],
  'name': 'Ytstenesdalen',
  'dangers': 'Steinsprang, elvegjøl, elveleie 100-530',
  'comment': 'Mellom 100-180 følger ruta en gammel vei/sti som går under store klippepartier. Det er sjelden mye snø i denne sida opp til 180.',
  'is_descent': True,
  'geojson': {'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'LineString',
    'coordinates': [[6.399579, 62.317338],
     [6.399515, 62.317527],
     [6.398721, 62.317478],
     [6.398849, 62.317278],
     [6.399193, 62.317079],
     [6.399193, 62.31662],
     [6.397347, 62.31671],
     [6.39636, 62.316102],
     [6.396232, 62.315703],
     [6.396317, 62.315225],
     [6.396532, 62.314577],
     [6.396532, 62.314008],
     [6.396682, 62.31347],
     [6.396832, 62.313061],
     [6.39709, 62.312313],
     [6.397079, 62.312029],
     [6.397294, 62.311855],
     [6.397401, 62.31166],
     [6.397208, 62.311471],
     [6.3973

In [48]:
class Peak(BaseModel):
  id: int
  name: str
  altitude: Optional[int]
  area: str
  


class Route(BaseModel):
  id: int
  name: str
  max_start_altitude: Optional[int]
  avalanche_start_zones: Optional[list] = []
  kast: Optional[int]
  dangers: Optional[str]
  exposure: Optional[str]
  difficulty: Optional[str]
  comment: Optional[str]
  equipment: Optional[str]
  max_steepness: Optional[int]
  is_descent: Optional[bool]
  parking_location: Optional[str]
  geojson: dict 
  aspect: list 
  peak: Peak = Field(None, alias="peak_id")
  distance_in_km: Optional[int]
  elevation: Optional[int]
  dry_snow: bool = False
  is_couloir: bool = False

  @validator("*", pre=True)
  def none_to_default(cls, value, field):
    if (value == "" or value == None):
      return field.default
    return value


In [49]:
valid_routes = []
invalid_routes = []

for route in routes:
  try:
    r = Route(**route)
    valid_routes.append(r)
  except ValidationError as e:
    error_text = (
      f'{route }: \n'
      f'{e} \n'
      f'-------------'
    )
    print(error_text)

    invalid_routes.append(route)

In [50]:
print(f'Invalid routes: {len(invalid_routes)}')
print(f'Valid routes: {len(routes)}')

Invalid routes: 0
Valid routes: 109


In [51]:
route_dicts = [route.dict() for route in valid_routes]
with open('routes.json', 'w') as file:
    file.write(json.dumps(route_dicts))